In [60]:
df_silver = spark.read.table("lh_bronze.brz_redata_demanda_evolucion_day")

display(df_silver.limit(5))
print(f"Total de filas iniciales: {df_silver.count():,}")

StatementMeta(, a7a408c1-284a-4036-a9d2-fddc8b9ef24b, 3, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, ef364af2-0f3b-4280-8c25-cd833411f47e)

Total de filas iniciales: 5,550


In [2]:
from pyspark.sql import functions as F

# eliminar columnas que tienen todas en nulo
# Contar cuántos valores no nulos tiene cada columna
non_null_counts = df_silver.select([
    F.count(F.col(c)).alias(c)
    for c in df_silver.columns
]).collect()[0].asDict()

# Filtrar las columnas que tienen al menos un valor no nulo
cols_to_keep = [c for c, count in non_null_counts.items() if count > 0]

# Crear nuevo DataFrame sin las columnas completamente nulas
df_no_null_cols = df_silver.select(cols_to_keep)

print(f"Columnas eliminadas: {[c for c in df_silver.columns if c not in cols_to_keep]}")
print(f"Total de columnas finales: {len(df_no_null_cols.columns)}")

# Mostrar el resultado
display(df_no_null_cols.limit(5))


StatementMeta(, a7a408c1-284a-4036-a9d2-fddc8b9ef24b, 4, Finished, Available, Finished)

Columnas eliminadas: ['ccaa_name', 'series_group_id', 'series_description', 'series_icon', 'series_attribute_type', 'series_magnitude']
Total de columnas finales: 14


SynapseWidget(Synapse.DataFrame, e27b81b2-5f04-4976-94c9-8b51d8072f39)

In [3]:
# eliminamos duplicados según todas las columnas excepto ingestion_timestamp
df_clean = df_no_null_cols.dropDuplicates([
    "geo_id", "geo_name", "geo_limit", "time_trunc",
    "series_type", "series_id", "series_title",
    "series_color", "is_composite", "series_last_update", "datetime", "value", "percentage"
])

display(df_clean.limit(5))
print(f"Total después de limpiar duplicados: {df_clean.count():,}")

StatementMeta(, a7a408c1-284a-4036-a9d2-fddc8b9ef24b, 5, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 43946bc1-109b-4938-a68d-04b262837e81)

Total después de limpiar duplicados: 3,710


In [4]:
# imprimimos valores únicos de cada columna
cols = ["geo_id", "geo_name", "geo_limit", "time_trunc", "series_type", "series_id", 
        "series_title", "series_color", "is_composite", "percentage"]

for c in cols:
    uniques = [row[c] for row in df_clean.select(c).distinct().collect()]
    count_uniques = df_clean.select(c).distinct().count()
    print(f"\n📌 Valores únicos en '{c}': {count_uniques}")
    print(uniques)

StatementMeta(, a7a408c1-284a-4036-a9d2-fddc8b9ef24b, 6, Finished, Available, Finished)


📌 Valores únicos en 'geo_id': 5
[8743, 8745, 8741, 8742, 8744]

📌 Valores únicos en 'geo_name': 5
['Comunidad de Ceuta', 'Islas Canarias', 'Islas Baleares', 'Comunidad de Melilla', 'Península']

📌 Valores únicos en 'geo_limit': 5
['ceuta', 'baleares', 'canarias', 'peninsular', 'melilla']

📌 Valores únicos en 'time_trunc': 1
['day']

📌 Valores únicos en 'series_type': 1
['Demanda']

📌 Valores únicos en 'series_id': 1
['10297']

📌 Valores únicos en 'series_title': 1
['Demanda']

📌 Valores únicos en 'series_color': 1
['#ffea00']

📌 Valores únicos en 'is_composite': 1
[False]

📌 Valores únicos en 'percentage': 1
[1.0]


In [5]:
# eliminamos las columnas que creamos no necesarios y/o que solo tengan 1 valor único
df_clean = df_clean.drop("time_trunc", 'series_type', 'series_id', 'series_title', "series_color", "ingestion_timestamp")

display(df_clean.limit(2))

StatementMeta(, a7a408c1-284a-4036-a9d2-fddc8b9ef24b, 7, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 4d98c18f-a5e2-4413-a2db-585a10da0524)

In [10]:
# eliminar filas que tengan nulos en columnas esenciales

# contamos filas
before_count = df_clean.count()

# eliminamos filas con nulos en columnas esenciales
df_clean = df_clean.na.drop(subset=["datetime", "value", "geo_name"])

# contamos filas tras la limpieza
after_count = df_clean.count()

if after_count < before_count:
    print(f"Se eliminaron {before_count - after_count:,} filas con valores nulos. Total filas tras la limpieza {after_count}.")
else:
    print("No se eliminaron filas; no había valores nulos en las columnas esenciales.")

StatementMeta(, a7a408c1-284a-4036-a9d2-fddc8b9ef24b, 12, Finished, Available, Finished)

No se eliminaron filas; no había valores nulos en las columnas esenciales.


In [11]:
# convertimos columnas al tipo correcto
df_clean = (df_clean
    .withColumn("datetime", F.to_timestamp("datetime"))
    .withColumn("value", F.col("value").cast("double"))
)

print("Finalizada la conversión de las columnas 'datetime' y 'value'.")

StatementMeta(, a7a408c1-284a-4036-a9d2-fddc8b9ef24b, 13, Finished, Available, Finished)

Finalizada la conversión de las columnas 'datetime' y 'value'.


In [12]:
from pyspark.sql import functions as F

# creamos nuevas columnas "year" y "month" desde "datetime"
df_clean = (
    df_clean
    .withColumn("year", F.year(F.col("datetime")))
    .withColumn("month", F.month(F.col("datetime")))
)

display(df_clean.limit(3))

StatementMeta(, a7a408c1-284a-4036-a9d2-fddc8b9ef24b, 14, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 0e6d1593-347b-4b3f-9cc2-f0204cc5daec)

In [13]:
# guardamos la tabla limpia en la capa Silver
df_clean.write.mode("overwrite").saveAsTable("lh_silver.slv_redata_demanda_evolucion_day_cleaned")

print("✅ Tabla guardada correctamente en lh_silver.slv_redata_demanda_evolucion_day_cleaned")

StatementMeta(, a7a408c1-284a-4036-a9d2-fddc8b9ef24b, 15, Finished, Available, Finished)

✅ Tabla guardada correctamente en lh_silver.slv_redata_demanda_evolucion_day_cleaned
